In [ ]:
import numpy as np
import os
import csv
from scipy.stats import poisson
from scipy.stats import norm
import pandas as pd
import itertools
import matplotlib.pyplot as plot

In [ ]:
def EstimateParameters(fixture_list_1, fixture_list_2, fixture_list_3,
                       teams, beta, thetapriormeans, thetapriorsds,
                       niter=1000, log=False, temp=0, zerooutinds=np.array([])):
    
    # xdata and ydata are coordinates and y values of data
    # xmodel are coordinates of model evaluations
    # thetaprior are prior guesses for parameters
    
    # draw initial
    if log:
        if hasattr(thetapriormeans, '__len__'):
            theta = np.zeros(len(thetapriormeans))
            for i in range(len(thetapriormeans)):
                theta[i] = np.exp(np.random.normal(thetapriormeans[i], thetapriorsds[i], 1))
        else:
            theta = np.exp(np.random.normal(thetapriormeans, thetapriorsds, 1))
    else:
        if hasattr(thetapriormeans, '__len__'):
            theta = np.zeros(len(thetapriormeans))
            for i in range(len(thetapriormeans)):
                theta[i] = np.random.normal(thetapriormeans[i], thetapriorsds[i], 1)
            # normalize
            #theta[(len(teams) + 1 - 1)] = -np.sum(theta[1:(len(teams) + 1 - 1)])
            #theta[((2 * len(teams)) + 1 - 1)] = -np.sum(theta[(len(teams) + 1):((2 * len(teams)) + 1 - 1)])
        else:
            theta = np.random.normal(thetapriormeans, thetapriorsds, 1)
    
    if hasattr(thetapriormeans, '__len__'):
        thetaarray = np.zeros((niter, len(thetapriormeans)))
    else:
        thetaarray = np.zeros(niter)
        
    accept_count = 0
    
    for j in range(niter):
        
        # temperature
        T = np.exp(-temp * ((i + 1) / niter))
        
        if log:
            if hasattr(thetapriormeans, '__len__'):
                thetastar = np.exp(np.log(theta) + np.random.normal(0, np.sqrt(beta), len(theta)))
            else:
                thetastar = np.exp(np.log(theta) + np.random.normal(0, np.sqrt(beta), 1))
        else:
            if hasattr(thetapriormeans, '__len__'):
                ind = np.random.normal(0, np.sqrt(beta), len(theta))
                # normalize
                #ind[(len(teams) + 1 - 1)] = -np.sum(ind[1:(len(teams) + 1 - 1)])
                #ind[((2 * len(teams)) + 1 - 1)] = -np.sum(ind[(len(teams) + 1):((2 * len(teams)) + 1 - 1)])
                thetastar = theta + ind
            else:
                ind = np.random.normal(0, np.sqrt(beta), 1)
                thetastar = theta + ind
        
        # get likelihood for each
        mu = theta[0]
        a = theta[1:(len(teams) + 1)]
        d = theta[(len(teams) + 1):((2 * len(teams)) + 1)]
        if len(zerooutinds) > 0:  # promoted team zero out
            a[zerooutinds] = 0
            d[zerooutinds] = 0
        a[0] = - np.sum(a[1:])  # normalize
        d[0] = - np.sum(d[1:])  # normalize
        alpha = theta[((2 * len(teams)) + 1)]
        Htheta = likelihood_three_seasons(fixture_list_1, fixture_list_2, fixture_list_3,
                                          teams, mu, a, d, alpha)
        
        mu = thetastar[0]
        a = thetastar[1:(len(teams) + 1)]
        d = thetastar[(len(teams) + 1):((2 * len(teams)) + 1)]
        if len(zerooutinds) > 0:  # promoted team zero out
            a[zerooutinds] = 0
            d[zerooutinds] = 0
        a[0] = - np.sum(a[1:])  # normalize
        d[0] = - np.sum(d[1:])  # normalize
        alpha = thetastar[((2 * len(teams)) + 1)]
        Hthetastar = likelihood_three_seasons(fixture_list_1, fixture_list_2, fixture_list_3,
                                              teams, mu, a, d, alpha)
        
        alpha = np.min([0, (1 / T) * (Hthetastar - Htheta)])
        
        # sample uniformly
        u = np.random.uniform(0, 1)
        
        # accept or not
        accept = np.log(u) <= alpha
        
        if accept:
            theta = thetastar
            accept_count += 1
            
        if hasattr(thetapriormeans, '__len__'):
            thetaarray[j, :] = theta
            if (j%10) == 0:
                print('------')
                print('Iteration: ', str(j))
                print('Home coefficient: '+str(thetaarray[j, 0]))
                print('Arsenal attack coefficient: '+str(thetaarray[j, 1]))
                print('acceptance ratio: ', accept_count / (j + 1))
        else:
            thetaarray[j] = theta
    
    # convert back and normalize
    if hasattr(thetapriormeans, '__len__'):
        if len(zerooutinds) > 0:  # zero out promoted teams
            thetaarray[:, (1 + zerooutinds).astype(int)] = 0.
            thetaarray[:, (1 + zerooutinds + len(teams)).astype(int)] = 0.
        thetaarray[:, 1] = - np.sum(thetaarray[:, 2:(len(teams) + 1)], axis=1)
        thetaarray[:, (len(teams) + 1)] = - np.sum(thetaarray[:, (len(teams) + 2):((2 * len(teams)) + 1)], axis=1)
    
    return thetaarray

# create likelihood eval for one game
def likelihood_one_game(goals_ht, goals_at, form_ht, form_at, mu, a_ht, d_ht, a_at, d_at, alpha):
    lambda_ht = np.exp(mu + a_ht + d_at + (alpha * form_ht))
    lambda_at = np.exp(a_at + d_ht + (alpha * form_at))
    p1 = poisson.pmf(goals_ht, lambda_ht)
    p2 = poisson.pmf(goals_at, lambda_at)
    return(p1 * p2)

# create likelihood eval for single season
def likelihood_season(fixtures_list, teams, mu, a, d, alpha):
    N = np.shape(fixtures_list)[0]
    goals_ht = fixtures_list[:, 2]
    goals_at = fixtures_list[:, 3]
    teams_ht = fixtures_list[:, 0]
    teams_at = fixtures_list[:, 1]
    
    teams_for_season = np.unique(teams_ht)
    
    points = np.zeros((38, 20))
    team_count = np.zeros(20)
    for i in range(N):
        points[team_count[np.where(teams_for_season == teams_ht[i])[0][0].astype(int)].astype(int), np.where(teams_for_season == teams_ht[i])[0][0].astype(int)] = (3 * (goals_ht[i] > goals_at[i])) + (goals_ht[i] == goals_at[i])
        points[team_count[np.where(teams_for_season == teams_at[i])[0][0].astype(int)].astype(int), np.where(teams_for_season == teams_at[i])[0][0].astype(int)] = (3 * (goals_ht[i] < goals_at[i])) + (goals_ht[i] == goals_at[i])
        team_count[np.where(teams_for_season == teams_ht[i])[0][0].astype(int)] += 1
        team_count[np.where(teams_for_season == teams_at[i])[0][0].astype(int)] += 1
    form = np.ones((38, 20)) * 7.5
    for j in range(20):
        form[5:, j] = np.cumsum(points[:, j])[5:] - np.cumsum(points[:, j])[:(38 - 5)]
    
    team_count = np.zeros(20)
    likelihood = np.zeros(N)
    for i in range(N):
        ind_ht = np.where(teams == teams_ht[i])[0][0].astype(int)
        ind_at = np.where(teams == teams_at[i])[0][0].astype(int)
        ind_for_season_ht = np.where(teams_for_season == teams_ht[i])[0][0].astype(int)
        ind_for_season_at = np.where(teams_for_season == teams_at[i])[0][0].astype(int)
        l = likelihood_one_game(goals_ht[i], goals_at[i],
                                form[team_count[ind_for_season_ht].astype(int), ind_for_season_ht].astype(int), form[team_count[ind_for_season_at].astype(int), ind_for_season_at],
                                mu, a[ind_ht], d[ind_ht], a[ind_at], d[ind_at], alpha)
        team_count[np.where(teams_for_season == teams_ht[i])[0][0].astype(int)] += 1
        team_count[np.where(teams_for_season == teams_at[i])[0][0].astype(int)] += 1
        likelihood[i] = l
    
    return(np.sum(np.log(likelihood)))

# likelihood over three seasons - weighted
def likelihood_three_seasons(fixture_list_1, fixture_list_2, fixture_list_3, teams, mu, a, d, alpha):
    likelihood = (0.2 * likelihood_season(fixture_list_1, teams, mu, a, d, alpha)) + (0.3 * likelihood_season(fixture_list_2, teams, mu, a, d, alpha)) + (0.5 * likelihood_season(fixture_list_3, teams, mu, a, d, alpha))
    return(likelihood)

# function to predict probabilities of fixtures
def predict_fixtures(new_fixtures, form, teams, mu, a, d, alpha, uncertainty=False):
    if uncertainty:
        # form is N x 2
        N = np.shape(new_fixtures)[0]
        teams_ht = new_fixtures[:, 0]
        teams_at = new_fixtures[:, 1]
        lambda_1 = np.zeros(N)
        lambda_2 = np.zeros(N)
        for i in range(N):
            muest = np.random.normal(mu[0], mu[1])
            aest = np.zeros(len(teams))
            dest = np.zeros(len(teams))
            for u in range(len(teams)):
                aest[u] = np.random.normal(a[u, 0], a[u, 1])
                dest[u] = np.random.normal(d[u, 0], d[u, 1])
            alphaest = np.random.normal(alpha[0], alpha[1])
            ind_ht = np.where(teams == teams_ht[i])[0][0].astype(int)
            ind_at = np.where(teams == teams_at[i])[0][0].astype(int)
            lambda_1[i] = np.exp(muest + aest[ind_ht] + dest[ind_at] + (alphaest * form[i, 0]))
            lambda_2[i] = np.exp(aest[ind_at] + dest[ind_ht] + (alphaest * form[i, 1]))
    else:
        # form is N x 2
        N = np.shape(new_fixtures)[0]
        teams_ht = new_fixtures[:, 0]
        teams_at = new_fixtures[:, 1]
        lambda_1 = np.zeros(N)
        lambda_2 = np.zeros(N)
        for i in range(N):
            ind_ht = np.where(teams == teams_ht[i])[0][0].astype(int)
            ind_at = np.where(teams == teams_at[i])[0][0].astype(int)
            lambda_1[i] = np.exp(mu + a[ind_ht] + d[ind_at] + (alpha * form[i, 0]))
            lambda_2[i] = np.exp(a[ind_at] + d[ind_ht] + (alpha * form[i, 1]))
    return(lambda_1, lambda_2)

def import_fixture_lists(filename_1, filename_2, filename_3):
    fixture_list_1 = pd.read_csv(filename_1, header=None)
    fixture_list_2 = pd.read_csv(filename_2, header=None)
    fixture_list_3 = pd.read_csv(filename_3, header=None)
    return(fixture_list_1, fixture_list_2, fixture_list_3)

def import_fixture_list(filename):
    fixture_list = pd.read_csv(filename, header=None)
    return(fixture_list)

In [ ]:
fixture_list_1, fixture_list_2, fixture_list_3 = import_fixture_lists("../data/prem_results_20162017.csv",
                                                                      "../data/prem_results_20172018.csv",
                                                                      "../data/prem_results_20182019.csv")

In [ ]:
# all teams in three seasons and fourth season
team_list = np.concatenate((np.unique(fixture_list_1.as_matrix()[:, 0]),
                            np.unique(fixture_list_2.as_matrix()[:, 0]),
                            np.unique(fixture_list_3.as_matrix()[:, 0]),
                            ((pd.read_csv("../data/team_id_20192020.csv", header=0)).as_matrix())[:, 0]))

In [ ]:
teams = np.unique(team_list)

In [ ]:
fl1 = fixture_list_1.as_matrix()
fl2 = fixture_list_2.as_matrix()
fl3 = fixture_list_3.as_matrix()

In [ ]:
# test
a = np.ones(len(teams)) * 0.1
d = np.ones(len(teams)) * 0.1
mu = 0.1
alpha = 0.1
l = likelihood_season(fl1, teams, mu, a, d, alpha)

In [ ]:
# test
predict_fixtures(fl1[1:3, :], np.array([[3, 3], [3, 3]]), teams, mu, a, d, alpha)

In [ ]:
priormeans = np.ones((2 * len(teams)) + 2) * 0.0
priorsds = np.ones((2 * len(teams)) + 2) * 0.1
beta = 0.00025
niter = 50000

# prior-out promoted teams
inds = list(np.where(teams == 'Norwich City')[0])
inds.append(np.where(teams == 'Aston Villa')[0][0])
inds.append(np.where(teams == 'Sheffield United')[0][0])
inds = np.array(inds)

est = EstimateParameters(fl1, fl2, fl3, teams, beta, priormeans, priorsds, niter=niter, zerooutinds=inds)

In [ ]:
tm = 'Tottenham'
plot.plot(est[:, (1 + np.where(teams == tm)[0][0]).astype(int)], 'r')
plot.plot(est[:, (len(teams) + 1 + np.where(teams == tm)[0][0]).astype(int)], 'b')
plot.show()

plot.plot(est[:, 0], 'k')
plot.show()

In [ ]:
# save parameters
burn = int((4 * niter) / 5)
mu_mean = np.array([np.mean(est[burn:, 0])])
mu_sd = np.array([np.std(est[burn:, 0])])
a_mean = np.mean(est[burn:, 1:(len(teams) + 1)], axis=0)
a_sd = np.std(est[burn:, 1:(len(teams) + 1)], axis=0)
d_mean = np.mean(est[burn:, (len(teams) + 1):((2 * len(teams)) + 1)], axis=0)
d_sd = np.std(est[burn:, (len(teams) + 1):((2 * len(teams)) + 1)], axis=0)
alpha_mean = np.array([np.mean(est[burn:, ((2 * len(teams)) + 1)])])
alpha_sd = np.array([np.std(est[burn:, ((2 * len(teams)) + 1)])])
means = np.concatenate((mu_mean, a_mean, d_mean, alpha_mean))
sds = np.concatenate((mu_sd, a_sd, d_sd, alpha_sd))

sds[1 + inds] = 0.1
sds[1 + len(teams) + inds] = 0.1

with open('../parameters/all_teams.csv', mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',')
    for tm in teams:
        csv_writer.writerow([tm])
csv_file.close()

with open('../parameters/all_teams_params.csv', mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',')
    for i in range(((2 * len(teams)) + 2)):
        csv_writer.writerow([means[i], sds[i]])
csv_file.close()

#### Save parameters here, and either predict for whole next season or individual gameweeks

In [ ]:
# create fixture list this season to predict
fixture_list_this_season = []
for i, fix in enumerate(list(itertools.permutations(((pd.read_csv("../data/team_id_20192020.csv", header=0)).as_matrix())[:, 0], 2))):
    fixture_list_this_season.append(list(fix))
fixture_list_this_season = np.array(fixture_list_this_season)

#### Update with week-by-week results 

In [ ]:
# pre-requisits: read 'teams', and function import_fixture_list

In [ ]:
# read this weeks results
fixture_list_this_week = import_fixture_list("../data/prem_results_20182019.csv")
fixture_list_this_week = fixture_list_this_week.loc[:10, :]

In [ ]:
# read params
all_teams_params = pd.read_csv("../parameters/all_teams_params.csv", header=None)

# particles
N = 1000
tp = all_teams_params.as_matrix()
params = np.zeros((np.shape(tp)[0], N))
for j in range(np.shape(tp)[0]):
    params[j, :] = np.random.normal(tp[j, 0], tp[j, 1], N)

In [ ]:
# particle filter likelihood
xi = np.zeros(N)
for i in range(N):
    for j in range(len(fixture_list_this_week.index)):
        a_ht = params[1 + np.where(teams == fixture_list_this_week.loc[fixture_list_this_week.index[j], 0])[0], i]
        a_at = params[1 + np.where(teams == fixture_list_this_week.loc[fixture_list_this_week.index[j], 1])[0], i]
        d_ht = params[1 + len(teams) + np.where(teams == fixture_list_this_week.loc[fixture_list_this_week.index[j], 0])[0], i]
        d_at = params[1 + len(teams) + np.where(teams == fixture_list_this_week.loc[fixture_list_this_week.index[j], 1])[0], i]
        xi[i] += np.log(likelihood_one_game(fixture_list_this_week.loc[fixture_list_this_week.index[j], 2],
                                            fixture_list_this_week.loc[fixture_list_this_week.index[j], 3],
                                            5, 5, params[0, i], a_ht, d_ht, a_at, d_at, params[-1, i]))

In [ ]:
resampled = np.random.choice(np.linspace(0, N - 1, N), N, p=np.exp(xi) / np.sum(np.exp(xi)))

In [ ]:
resampled_params = params[:, resampled.astype(int)]

In [ ]:
new_means = np.mean(resampled_params, axis=1)
new_sds = np.std(resampled_params, axis=1)

In [ ]:
#with open('../parameters/all_teams_params.csv', mode='w', newline='') as csv_file:
#    csv_writer = csv.writer(csv_file, delimiter=',')
#    for i in range(((2 * len(teams)) + 2)):
#        csv_writer.writerow([new_means[i], new_sds[i]])
#csv_file.close()

#### Simulate seasons performance

In [ ]:
wins = np.zeros(20)
topfour = np.zeros(20)
niter = 1000

# predict lambdas - average form
muest = np.hstack((means[0], sds[0]))
aest = np.hstack((np.reshape(means[1:(1+len(teams))], ((len(teams), 1))),
                  np.reshape(sds[1:(1+len(teams))], ((len(teams), 1)))))
dest = np.hstack((np.reshape(means[(1+len(teams)):(1+(2*len(teams)))], ((len(teams), 1))),
                  np.reshape(sds[(1+len(teams)):(1+(2*len(teams)))], ((len(teams), 1)))))
alphaest = np.hstack((means[((2 * len(teams)) + 1)], sds[((2 * len(teams)) + 1)]))

for j in range(niter):
    # work out total goals predicted
    total_goals = np.zeros(20)
    #actual_goals = np.zeros(20)
    total_goals_conceded = np.zeros(20)
    #actual_goals_conceded = np.zeros(20)
    tms = np.unique(fixture_list_this_season[:, 0])
    goals_this_season = predict_fixtures(fixture_list_this_season, np.ones((380, 2)) * 5, teams, muest, aest, dest, alphaest, uncertainty=True)
    pnts = np.zeros(20)
    for i in range(np.shape(fixture_list_this_season)[0]):
        goals_ht = np.random.poisson(goals_this_season[0][i])
        goals_at = np.random.poisson(goals_this_season[1][i])
        total_goals[np.where(tms == fixture_list_this_season[i, 0])[0][0].astype(int)] += goals_ht
        total_goals[np.where(tms == fixture_list_this_season[i, 1])[0][0].astype(int)] += goals_at
        pnts[np.where(tms == fixture_list_this_season[i, 0])[0][0].astype(int)] += ((goals_at < goals_ht) * 3) + (goals_at == goals_ht)
        pnts[np.where(tms == fixture_list_this_season[i, 1])[0][0].astype(int)] += ((goals_at > goals_ht) * 3) + (goals_at == goals_ht)
        #actual_goals[np.where(tms == fl2[i, 0])[0][0].astype(int)] += fl2[i, 2]
        #actual_goals[np.where(tms == fl2[i, 1])[0][0].astype(int)] += fl2[i, 3]
        total_goals_conceded[np.where(tms == fixture_list_this_season[i, 0])[0][0].astype(int)] += goals_at
        total_goals_conceded[np.where(tms == fixture_list_this_season[i, 1])[0][0].astype(int)] += goals_ht
        #actual_goals_conceded[np.where(tms == fl2[i, 0])[0][0].astype(int)] += fl2[i, 3]
        #actual_goals_conceded[np.where(tms == fl2[i, 1])[0][0].astype(int)] += fl2[i, 2]
    # update title wins
    wins[np.argmax(pnts)] += 1
    # update top four finishes
    topfour[np.argsort(pnts)[16:]] += 1

In [ ]:
ind = np.arange(len(topfour))  # the x locations for the groups
width = 0.35  # the width of the bars

fig, ax = plot.subplots()
rects1 = ax.bar(ind - width/2, wins/niter, width,
                label='Wins')
rects2 = ax.bar(ind + width/2, topfour/niter, width,
                label='Top Four')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Probability')
ax.set_xticks(ind)
ax.set_xticklabels(tuple(tms))
ax.legend()

In [ ]:
print(tms)
print(wins/niter)

In [ ]:
print(tms)
print(total_goals)
print(total_goals_conceded)
print(total_goals - total_goals_conceded)
print(np.argsort(pnts))